## 🌐 텍스트 임베딩 모델

### ℹ️ 정보
텍스트 임베딩 모델 제공자와의 내장 통합에 대한 문서는 통합 섹션에서 확인하세요.

`Embeddings` 클래스는 텍스트 임베딩 모델과의 인터페이스를 위해 설계된 클래스입니다. 많은 임베딩 모델 제공자들(OpenAI, Cohere, Hugging Face 등)이 있으며, 이 클래스는 모든 제공자들에 대해 표준 인터페이스를 제공하도록 설계되었습니다.

임베딩은 텍스트의 벡터 표현을 생성합니다. 이는 텍스트를 벡터 공간에서 생각할 수 있게 하며, 벡터 공간에서 가장 유사한 텍스트 조각을 찾는 의미적 검색과 같은 작업을 할 수 있다는 것을 의미합니다.

LangChain에서 기본 `Embeddings` 클래스는 두 가지 메소드를 제공합니다: 하나는 문서를 임베딩하기 위한 것이고, 다른 하나는 쿼리를 임베딩하기 위한 것입니다. 전자는 여러 텍스트를 입력으로 받으며, 후자는 단일 텍스트를 받습니다. 이를 두 가지 별도의 메소드로 제공하는 이유는 일부 임베딩 제공자들이 문서(검색 대상)와 쿼리(검색 쿼리 자체)에 대해 서로 다른 임베딩 방법을 가지고 있기 때문입니다.


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key="...")

In [ ]:
# 메시지를 리스트 형태에서 임베딩 벡터로 변환
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

In [ ]:
#쿼리 임베딩
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

#### 캐시백엔드임베딩 : 임베딩값을 저장시켜 같은값이 들어오면 추가적으로 실행하지 않고, 저장된 벡터에서 값을 반환

In [ ]:

from langchain.embeddings import CacheBackedEmbeddings

from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

underlying_embeddings = OpenAIEmbeddings()

store = LocalFileStore("./cache/") #캐쉬파일로 저장을 하면

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

# store 를 토대로 임베딩 캐쉬를 통해 같은 내용이 들어오면 캐쉬된 값을 반환

# Vector Stores

In [1]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = 'your api_key'

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('/Users/kdb/Desktop/psedo_labs/4weeks_retrieval/data/datas.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OpenAIEmbeddings())



# Document 로드하고 text spliter로 나누고 임베딩을 하고 vector store에 로드

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
query = "What did the president say about Ketanji Brown Jackson" #쿼리를 넣어서 유사도 검색
docs = db.similarity_search(query)
print(docs[0].page_content)

In [ ]:
embedding_vector = OpenAIEmbeddings().embed_query(query) #쿼리를 벡터로 변환하여 보다 정확한 검색
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

In [ ]:
from langchain_community.vectorstores import Qdrant


db = await Qdrant.afrom_documents(documents, embeddings, "http://localhost:6333") #비동기로 동작

query = "What did the president say about Ketanji Brown Jackson" 
docs = await db.asimilarity_search(query)
print(docs[0].page_content)

embedding_vector = embeddings.embed_query(query)
docs = await db.asimilarity_search_by_vector(embedding_vector)



Maximum marginal relevance search (MMR)
- 최대 한계 관련성은 쿼리와의 유사성 및 선택한 문서 간의 다양성을 최적화합니다. 비동기 API에서도 지원됩니다.

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = await qdrant.amax_marginal_relevance_search(query, k=2, fetch_k=10) #최대 한계 관련성
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

### Custom Retrieval

In [ ]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List


class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]: # 쿼리를 받고, *는 키워드 인자를 받는다는 의미, run_manager는 콜백 매니저 
        return [Document(page_content=query)] # 결과를 반환

retriever = CustomRetriever()

retriever.get_relevant_documents("bar")